In [2]:
######################################################
## RNN/ 1D CNN Data Preparation (25 Oct 2019 Data) ##
##########################################################

In [4]:
import pandas as pd
import numpy as np
store_25_Oct = pd.read_hdf("D:/CERN/data/singlepi_e100GeV_pu200_25Oct/singlepi_e100GeV_pu200.h5")
print(len(store_25_Oct))

32274


In [6]:
# Normalize the Data
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()

store_25_Oct_normalized = store_25_Oct.copy()

store_25_Oct_normalized['E'] = scaler.fit_transform(store_25_Oct_normalized[["E"]])
store_25_Oct_normalized['eta'] = scaler.fit_transform(store_25_Oct_normalized[["eta"]])
store_25_Oct_normalized['phi'] = scaler.fit_transform(store_25_Oct_normalized[["phi"]])
store_25_Oct_normalized['x'] = scaler.fit_transform(store_25_Oct_normalized[["x"]])
store_25_Oct_normalized['y'] = scaler.fit_transform(store_25_Oct_normalized[["y"]])
store_25_Oct_normalized['z'] = scaler.fit_transform(store_25_Oct_normalized[["z"]])
store_25_Oct_normalized['nHits'] = scaler.fit_transform(store_25_Oct_normalized[["nHits"]])

In [7]:
store_25_Oct_normalized.head()

,event,trackster,purity,layer,E,eta,phi,x,y,z,nHits,trckEn,trckEta,trckPhi,trckType
0,1.0,0.0,0.0,1.0,-0.223694,0.089506,0.424994,-0.187758,0.410803,-0.858269,0.238624,57.759506,1.897144,0.196742,211.0
1,1.0,0.0,0.0,1.0,-0.175444,-0.148224,0.639593,-0.082362,0.633005,-0.858269,0.731336,57.759506,1.897144,0.196742,211.0
2,1.0,0.0,0.0,1.0,-0.220151,-0.144319,0.396566,0.012880,0.397149,-0.858269,-0.500445,57.759506,1.897144,0.196742,211.0
3,1.0,0.0,0.0,1.0,-0.268674,-0.007692,1.003738,-0.420212,0.957504,-0.858269,-0.500445,57.759506,1.897144,0.196742,211.0
4,1.0,0.0,0.0,1.0,-0.213881,-0.298404,0.989808,-0.179869,0.984447,-0.858269,-0.500445,57.759506,1.897144,0.196742,211.0


In [8]:
store_25_Oct_normalized.describe()

,event,trackster,purity,layer,E,eta,phi,x,y,z,nHits,trckEn,trckEta,trckPhi,trckType
count,32274.000000,32274.000000,32274.000000,32274.000000,3.227400e+04,3.227400e+04,3.227400e+04,3.227400e+04,3.227400e+04,3.227400e+04,3.227400e+04,32274.000000,32274.000000,32274.000000,32274.0
mean,50.440416,0.476576,0.453337,12.682314,-6.267665e-18,2.205964e-16,1.675861e-16,-1.641427e-16,2.961964e-15,1.666883e-16,-1.109915e-15,70.012738,1.911353,0.017087,211.0
std,30.650939,1.032289,0.799332,9.393734,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,35.154862,0.042011,0.331789,0.0
min,1.000000,0.000000,0.000000,1.000000,-2.781280e-01,-3.351729e+00,-2.570699e+00,-2.997471e+00,-2.541636e+00,-8.582701e-01,-7.468005e-01,0.287669,1.657507,-0.696673,211.0
25%,20.000000,0.000000,0.000000,6.000000,-2.524945e-01,-6.471880e-01,-8.326390e-01,-6.764332e-01,-8.328407e-01,-5.550141e-01,-5.004445e-01,53.323616,1.882692,-0.288373,211.0
50%,51.000000,0.000000,0.000000,10.000000,-2.201506e-01,-1.988175e-01,-1.338764e-01,4.753547e-03,-1.281237e-01,-2.933657e-01,-2.540885e-01,72.709122,1.907120,-0.059937,211.0
75%,79.000000,1.000000,1.000000,17.000000,-1.294619e-01,5.766584e-01,8.173580e-01,6.513461e-01,8.321881e-01,1.883302e-01,-7.732498e-03,94.339554,1.933139,0.303409,211.0
max,100.000000,8.000000,2.000000,50.000000,3.602063e+01,5.239983e+00,2.786753e+00,5.566643e+00,3.111594e+00,7.548292e+00,1.255642e+01,151.615540,2.026844,0.629487,211.0


In [9]:
max(store_25_Oct_normalized[["x"]])

'x'

In [10]:
max(store_25_Oct_normalized["x"])

5.566643027811152

In [11]:
min(store_25_Oct_normalized["x"])

-2.997470674050745

In [16]:
# The following method finds/returns the index of 'n' closest neighbours in a given data-frame('frame') with respect to a 
# given point('point')  in x,y coordinate system. Closeness is based on Euclidean Distance.

def closestNeighbours(frame,point,n):
    frame['distance'] = np.linalg.norm(frame[['x', 'y','z']].sub(np.array(point)), axis=1)
    frame.sort_values(by='distance',inplace = True)
    return list(frame.index[:n])

In [39]:
# for all the event,Generate sequences of atleast size 3 . Sequences are generated based on event and trackster.
# Consider all the possible layer cluster from preceding and successive layers.
# Don't consider the first and last layer in a trackster.
# sequence_id => This field is the sequence identifier. for example, sequence_id == 1_0_1 should be decoded as follows:
# 1_0_1 => (eventId_tracksterId_sequenceCount)

def generateSequence_with_all_possible_layers(input_df,seq_len = 3):
    if (seq_len < 3 or (seq_len % 2) == 0 ):
        print ("Sequence length should be an odd number that must be greater than or equal to 3.")
        return
    #initialize the sequence store
    seq_df_columns=['index_original','sequence_id']
    seq_df_columns.extend(list(input_df.columns))
    seq_df = pd.DataFrame(columns = seq_df_columns )
    
    # Find the min, max event id in the df
    min_event_id , max_event_id  = int(min(input_df.event)) , int(max(input_df.event))
    
    # Assume sequence_size to be an odd number. Number of nearest neighbors to look from previous and successive layers
    num_neighbours = int((seq_len - 1 ) / 2 )
    print('num_neighbours: {}'.format(num_neighbours))
    for event in range(min_event_id, max_event_id + 1):
        df = input_df[input_df.event == event].copy()
        unique_tracksters_event = df.trackster.unique()
        for trackster in unique_tracksters_event:
            df_trackster = df[df.trackster == trackster].copy()
            count = 1
            min_layer,max_layer = min(df_trackster.layer),max(df_trackster.layer)
            for row in df_trackster.itertuples():
                event_id = row[1]
                trackster_id = row[2] 
                layer_id = row[4]
                x = row[8]
                y = row[9]
                z = row[10]
                if (layer_id == min_layer or layer_id == max_layer):
                    continue
                else:
                    # check if sequence can be generated
                    total_records_in_prev_layers = sum(df_trackster.layer[df_trackster.layer < layer_id ].value_counts())
                    total_records_in_next_layers = sum(df_trackster.layer[df_trackster.layer > layer_id ].value_counts())
                    if (total_records_in_prev_layers < num_neighbours or total_records_in_next_layers < num_neighbours):
                        print ('Sequence cannt be generated for event_id : {0} , trackster_id : {1} ,layer_id : {2}.'.format(event_id, trackster_id, layer_id))
                        continue
                    sequence_id = str(int(event_id)) + '_' + str(int(trackster_id)) + '_' + str(count)
                    point = list([x,y,z])
                    
                    prev_layers =   df_trackster.layer[df_trackster.layer < layer_id ].tolist() 
                    prev_layer_df = df_trackster[df_trackster.layer.isin (prev_layers)].copy()
                    prev_index = closestNeighbours(prev_layer_df,point,num_neighbours)
                    prev_rows = df_trackster.loc[prev_index, :].copy()
                    prev_rows['index_original']= prev_index            
                    prev_rows['sequence_id']= sequence_id
                    prev_rows = prev_rows[['index_original', 'sequence_id', 'event', 'trackster', 'purity', 'layer', 'E', 'eta', 'phi', 'x', 'y','z','nHits', 'trckEn', 'trckEta','trckPhi','trckType']]
                    prev_rows.sort_values(by='index_original',inplace = True)

                    seq_df = seq_df.append(prev_rows, ignore_index=True)
                  

                    current_row = list([row[0]])
                    current_row.append(sequence_id)
                    current_row.extend(row[1:])
                    seq_df.loc[len(seq_df)] = current_row

                    next_layer =   df_trackster.layer[df_trackster.layer > layer_id ].tolist()
                    next_layer_df = df_trackster[df_trackster.layer.isin (next_layer)].copy()
                    next_index = closestNeighbours(next_layer_df,point,num_neighbours)
                    next_rows = df_trackster.loc[next_index, :].copy()
                    next_rows['index_original']= next_index
                    next_rows['sequence_id']= sequence_id
                    next_rows = next_rows[['index_original', 'sequence_id', 'event', 'trackster', 'purity', 'layer', 'E', 'eta', 'phi', 'x', 'y','z','nHits', 'trckEn', 'trckEta','trckPhi','trckType']]
                    next_rows.sort_values(by='index_original',inplace = True)
                    seq_df = seq_df.append(next_rows, ignore_index=True)
                    count = count + 1
    print('len(seq_df): {}'.format(len(seq_df)))
    return seq_df.copy()


In [19]:
seq_store_25_Oct_normalized = generateSequence_with_all_possible_layers(store_25_Oct_normalized,seq_len = 3)

len(seq_df): 91716


In [22]:
seq_store_25_Oct_normalized.head(6)

,index_original,sequence_id,event,trackster,purity,layer,E,eta,phi,x,y,z,nHits,trckEn,trckEta,trckPhi,trckType
0,0,1_0_1,1.0,0.0,0.0,1.0,-0.223694,0.089506,0.424994,-0.187758,0.410803,-0.858269,0.238624,57.759506,1.897144,0.196742,211.0
1,9,1_0_1,1.0,0.0,0.0,2.0,-0.200869,0.185687,0.620558,-0.324302,0.587929,-0.816664,-0.254089,57.759506,1.897144,0.196742,211.0
2,23,1_0_1,1.0,0.0,0.0,3.0,-0.262887,0.094513,0.684267,-0.247567,0.659124,-0.727444,-0.500445,57.759506,1.897144,0.196742,211.0
3,1,1_0_2,1.0,0.0,0.0,1.0,-0.175444,-0.148224,0.639593,-0.082362,0.633005,-0.858269,0.731336,57.759506,1.897144,0.196742,211.0
4,10,1_0_2,1.0,0.0,0.0,2.0,-0.196511,-0.073152,0.633177,-0.123334,0.622039,-0.816664,-0.254089,57.759506,1.897144,0.196742,211.0
5,32,1_0_2,1.0,0.0,0.0,3.0,-0.264898,-0.187583,0.670189,-0.010778,0.672387,-0.727444,-0.500445,57.759506,1.897144,0.196742,211.0


In [21]:
len(seq_store_25_Oct_normalized)

91716

In [23]:
seq_store_25_Oct = generateSequence_with_all_possible_Layers(store_25_Oct,seq_len = 3)

len(seq_df): 91716


In [24]:
seq_store_25_Oct.head(6)

,index_original,sequence_id,event,trackster,purity,layer,E,eta,phi,x,y,z,nHits,trckEn,trckEta,trckPhi,trckType
0,1,1_0_1,1.0,0.0,0.0,1.0,0.134952,1.931620,0.233432,92.770134,22.057596,322.102753,7.0,57.759506,1.897144,0.196742,211.0
1,9,1_0_1,1.0,0.0,0.0,2.0,0.104475,1.976022,0.226741,88.978676,20.528135,323.047272,3.0,57.759506,1.897144,0.196742,211.0
2,23,1_0_1,1.0,0.0,0.0,3.0,0.030137,1.963898,0.249134,90.181198,22.943840,325.072754,2.0,57.759506,1.897144,0.196742,211.0
3,1,1_0_2,1.0,0.0,0.0,1.0,0.134952,1.931620,0.233432,92.770134,22.057596,322.102753,7.0,57.759506,1.897144,0.196742,211.0
4,10,1_0_2,1.0,0.0,0.0,2.0,0.109699,1.941603,0.231176,92.128059,21.685528,323.047272,3.0,57.759506,1.897144,0.196742,211.0
5,23,1_0_2,1.0,0.0,0.0,3.0,0.030137,1.963898,0.249134,90.181198,22.943840,325.072754,2.0,57.759506,1.897144,0.196742,211.0


In [48]:
def generateSequence_with_only_immediate_layers(input_df,seq_len = 3):
    if (len(input_df) < 3 or seq_len < 3 or (seq_len % 2) == 0 ):
        print ("Sequence length should be an odd number that must be greater than or equal to 3.")
        return
    #initialize the sequence store
    seq_df_columns=['index_original','sequence_id']
    seq_df_columns.extend(list(input_df.columns))
    seq_df = pd.DataFrame(columns = seq_df_columns )
    
    # Find the min, max event id in the df
    min_event_id , max_event_id  = int(min(input_df.event)) , int(max(input_df.event))
    
    # Assume sequence_size to be an odd number. Number of nearest neighbors to look from previous and successive layers
    num_neighbours = int((seq_len - 1 ) / 2 )
    print('num_neighbours: {}'.format(num_neighbours))
    
    for event in range(min_event_id, max_event_id + 1):
        df = input_df[input_df.event == event].copy()
        unique_tracksters_event = df.trackster.unique()
        for trackster in unique_tracksters_event:
            df_trackster = df[df.trackster == trackster].copy()
            count = 1
            min_layer,max_layer = min(df_trackster.layer),max(df_trackster.layer)
            for row in df_trackster.itertuples():
                event_id = row[1]
                trackster_id = row[2] 
                layer_id = row[4]
                x = row[8]
                y = row[9]
                z = row[10]
                if (layer_id == min_layer or layer_id == max_layer):
                    continue
                else:
                    # check if sequence can be generated
                    total_records_in_prev_layers = sum(df_trackster.layer[df_trackster.layer == layer_id - 1.0 ].value_counts())
                    total_records_in_next_layers = sum(df_trackster.layer[df_trackster.layer == layer_id + 1.0 ].value_counts())
                    if (total_records_in_prev_layers < num_neighbours or total_records_in_next_layers < num_neighbours):
                        print ('Sequence cannt be generated for event_id : {0} , trackster_id : {1} ,layer_id : {2}.'.format(event_id, trackster_id, layer_id))
                        continue
                    sequence_id = str(int(event_id)) + '_' + str(int(trackster_id)) + '_' + str(count)
                    point = list([x,y,z])
                    
                    prev_layer_df = df_trackster[df_trackster.layer == (layer_id - 1.0)].copy()
                    prev_index = closestNeighbours(prev_layer_df,point,num_neighbours)
                    prev_rows = df_trackster.loc[prev_index, :].copy()
                    prev_rows['index_original']= prev_index            
                    prev_rows['sequence_id']= sequence_id
                    prev_rows = prev_rows[['index_original', 'sequence_id', 'event', 'trackster', 'purity', 'layer', 'E', 'eta', 'phi', 'x', 'y','z','nHits', 'trckEn', 'trckEta','trckPhi','trckType']]
                    prev_rows.sort_values(by='index_original',inplace = True)

                    seq_df = seq_df.append(prev_rows, ignore_index=True)                   
    
                    current_row = list([row[0]])
                    current_row.append(sequence_id)
                    current_row.extend(row[1:])
                    seq_df.loc[len(seq_df)] = current_row

                    
                    next_layer_df = df_trackster[df_trackster.layer == (layer_id + 1.0)].copy()
                    next_index = closestNeighbours(next_layer_df,point,num_neighbours)
                    next_rows = df_trackster.loc[next_index, :].copy()
                    next_rows['index_original']= next_index
                    next_rows['sequence_id']= sequence_id
                    next_rows = next_rows[['index_original', 'sequence_id', 'event', 'trackster', 'purity', 'layer', 'E', 'eta', 'phi', 'x', 'y','z','nHits', 'trckEn', 'trckEta','trckPhi','trckType']]
                    next_rows.sort_values(by='index_original',inplace = True)
                    seq_df = seq_df.append(next_rows, ignore_index=True)
                    count = count + 1
    print('len(seq_df): {}'.format(len(seq_df)))
    return seq_df.copy()


In [45]:
immediate_layer_seq_store_25_Oct = generateSequence_with_only_immediate_layers(store_25_Oct,seq_len = 3)

num_neighbours: 1
len(seq_df): 90324


In [46]:
immediate_layer_seq_store_25_Oct.head(12)

,index_original,sequence_id,event,trackster,purity,layer,E,eta,phi,x,y,z,nHits,trckEn,trckEta,trckPhi,trckType
0,1,1_0_1,1.0,0.0,0.0,1.0,0.134952,1.931620,0.233432,92.770134,22.057596,322.102753,7.0,57.759506,1.897144,0.196742,211.0
1,9,1_0_1,1.0,0.0,0.0,2.0,0.104475,1.976022,0.226741,88.978676,20.528135,323.047272,3.0,57.759506,1.897144,0.196742,211.0
2,23,1_0_1,1.0,0.0,0.0,3.0,0.030137,1.963898,0.249134,90.181198,22.943840,325.072754,2.0,57.759506,1.897144,0.196742,211.0
3,1,1_0_2,1.0,0.0,0.0,1.0,0.134952,1.931620,0.233432,92.770134,22.057596,322.102753,7.0,57.759506,1.897144,0.196742,211.0
4,10,1_0_2,1.0,0.0,0.0,2.0,0.109699,1.941603,0.231176,92.128059,21.685528,323.047272,3.0,57.759506,1.897144,0.196742,211.0
5,23,1_0_2,1.0,0.0,0.0,3.0,0.030137,1.963898,0.249134,90.181198,22.943840,325.072754,2.0,57.759506,1.897144,0.196742,211.0
6,2,1_0_3,1.0,0.0,0.0,1.0,0.081363,1.932140,0.148012,94.262695,14.054753,322.102753,2.0,57.759506,1.897144,0.196742,211.0
7,11,1_0_3,1.0,0.0,0.0,2.0,0.251543,1.947605,0.158610,92.875359,14.855724,323.047272,5.0,57.759506,1.897144,0.196742,211.0
8,24,1_0_3,1.0,0.0,1.0,3.0,0.998739,1.911525,0.137086,97.353493,13.430076,325.072754,16.0,57.759506,1.897144,0.196742,211.0
9,3,1_0_4,1.0,0.0,0.0,1.0,0.023200,1.950308,0.361422,87.475647,33.068218,322.102753,2.0,57.759506,1.897144,0.196742,211.0


In [47]:
# Lets save the sequence of length 3

immediate_layer_seq_store_25_Oct.to_csv('D:/CERN/data/singlepi_e100GeV_pu200_25Oct/L3_immediate_layer_seq_store_25_Oct.csv',index = False)

In [49]:
immediate_layer_seq_store_25_Oct_len_5 = generateSequence_with_only_immediate_layers(store_25_Oct,seq_len = 5)

num_neighbours: 2
Sequence cannt be generated for event_id : 1.0 , trackster_id : 0.0 ,layer_id : 29.0.
Sequence cannt be generated for event_id : 1.0 , trackster_id : 0.0 ,layer_id : 29.0.
Sequence cannt be generated for event_id : 1.0 , trackster_id : 0.0 ,layer_id : 29.0.
Sequence cannt be generated for event_id : 1.0 , trackster_id : 0.0 ,layer_id : 30.0.
Sequence cannt be generated for event_id : 1.0 , trackster_id : 0.0 ,layer_id : 31.0.
Sequence cannt be generated for event_id : 1.0 , trackster_id : 0.0 ,layer_id : 32.0.
Sequence cannt be generated for event_id : 1.0 , trackster_id : 0.0 ,layer_id : 32.0.
Sequence cannt be generated for event_id : 1.0 , trackster_id : 0.0 ,layer_id : 34.0.
Sequence cannt be generated for event_id : 1.0 , trackster_id : 0.0 ,layer_id : 34.0.
Sequence cannt be generated for event_id : 1.0 , trackster_id : 0.0 ,layer_id : 34.0.
Sequence cannt be generated for event_id : 1.0 , trackster_id : 0.0 ,layer_id : 34.0.
Sequence cannt be generated for even

Sequence cannt be generated for event_id : 5.0 , trackster_id : 0.0 ,layer_id : 34.0.
Sequence cannt be generated for event_id : 5.0 , trackster_id : 0.0 ,layer_id : 34.0.
Sequence cannt be generated for event_id : 5.0 , trackster_id : 0.0 ,layer_id : 35.0.
Sequence cannt be generated for event_id : 5.0 , trackster_id : 0.0 ,layer_id : 36.0.
Sequence cannt be generated for event_id : 5.0 , trackster_id : 0.0 ,layer_id : 37.0.
Sequence cannt be generated for event_id : 5.0 , trackster_id : 0.0 ,layer_id : 38.0.
Sequence cannt be generated for event_id : 5.0 , trackster_id : 1.0 ,layer_id : 2.0.
Sequence cannt be generated for event_id : 5.0 , trackster_id : 1.0 ,layer_id : 2.0.
Sequence cannt be generated for event_id : 5.0 , trackster_id : 1.0 ,layer_id : 2.0.
Sequence cannt be generated for event_id : 5.0 , trackster_id : 1.0 ,layer_id : 3.0.
Sequence cannt be generated for event_id : 5.0 , trackster_id : 1.0 ,layer_id : 4.0.
Sequence cannt be generated for event_id : 5.0 , trackster_

Sequence cannt be generated for event_id : 9.0 , trackster_id : 0.0 ,layer_id : 33.0.
Sequence cannt be generated for event_id : 9.0 , trackster_id : 0.0 ,layer_id : 33.0.
Sequence cannt be generated for event_id : 9.0 , trackster_id : 0.0 ,layer_id : 33.0.
Sequence cannt be generated for event_id : 9.0 , trackster_id : 0.0 ,layer_id : 35.0.
Sequence cannt be generated for event_id : 9.0 , trackster_id : 0.0 ,layer_id : 36.0.
Sequence cannt be generated for event_id : 9.0 , trackster_id : 0.0 ,layer_id : 37.0.
Sequence cannt be generated for event_id : 9.0 , trackster_id : 1.0 ,layer_id : 26.0.
Sequence cannt be generated for event_id : 9.0 , trackster_id : 1.0 ,layer_id : 27.0.
Sequence cannt be generated for event_id : 9.0 , trackster_id : 1.0 ,layer_id : 27.0.
Sequence cannt be generated for event_id : 9.0 , trackster_id : 1.0 ,layer_id : 27.0.
Sequence cannt be generated for event_id : 9.0 , trackster_id : 1.0 ,layer_id : 28.0.
Sequence cannt be generated for event_id : 9.0 , track

Sequence cannt be generated for event_id : 13.0 , trackster_id : 5.0 ,layer_id : 6.0.
Sequence cannt be generated for event_id : 13.0 , trackster_id : 6.0 ,layer_id : 5.0.
Sequence cannt be generated for event_id : 13.0 , trackster_id : 6.0 ,layer_id : 6.0.
Sequence cannt be generated for event_id : 13.0 , trackster_id : 6.0 ,layer_id : 6.0.
Sequence cannt be generated for event_id : 13.0 , trackster_id : 6.0 ,layer_id : 6.0.
Sequence cannt be generated for event_id : 13.0 , trackster_id : 6.0 ,layer_id : 8.0.
Sequence cannt be generated for event_id : 13.0 , trackster_id : 6.0 ,layer_id : 8.0.
Sequence cannt be generated for event_id : 13.0 , trackster_id : 6.0 ,layer_id : 9.0.
Sequence cannt be generated for event_id : 13.0 , trackster_id : 6.0 ,layer_id : 9.0.
Sequence cannt be generated for event_id : 13.0 , trackster_id : 6.0 ,layer_id : 9.0.
Sequence cannt be generated for event_id : 13.0 , trackster_id : 6.0 ,layer_id : 9.0.
Sequence cannt be generated for event_id : 13.0 , trac

Sequence cannt be generated for event_id : 15.0 , trackster_id : 2.0 ,layer_id : 27.0.
Sequence cannt be generated for event_id : 15.0 , trackster_id : 2.0 ,layer_id : 27.0.
Sequence cannt be generated for event_id : 16.0 , trackster_id : 0.0 ,layer_id : 34.0.
Sequence cannt be generated for event_id : 16.0 , trackster_id : 0.0 ,layer_id : 34.0.
Sequence cannt be generated for event_id : 16.0 , trackster_id : 1.0 ,layer_id : 2.0.
Sequence cannt be generated for event_id : 16.0 , trackster_id : 1.0 ,layer_id : 2.0.
Sequence cannt be generated for event_id : 16.0 , trackster_id : 1.0 ,layer_id : 2.0.
Sequence cannt be generated for event_id : 16.0 , trackster_id : 1.0 ,layer_id : 2.0.
Sequence cannt be generated for event_id : 16.0 , trackster_id : 1.0 ,layer_id : 4.0.
Sequence cannt be generated for event_id : 16.0 , trackster_id : 1.0 ,layer_id : 4.0.
Sequence cannt be generated for event_id : 16.0 , trackster_id : 1.0 ,layer_id : 4.0.
Sequence cannt be generated for event_id : 16.0 , 

...

Sequence cannt be generated for event_id : 83.0 , trackster_id : 0.0 ,layer_id : 43.0.
Sequence cannt be generated for event_id : 83.0 , trackster_id : 0.0 ,layer_id : 43.0.
Sequence cannt be generated for event_id : 83.0 , trackster_id : 0.0 ,layer_id : 43.0.
Sequence cannt be generated for event_id : 83.0 , trackster_id : 0.0 ,layer_id : 43.0.
Sequence cannt be generated for event_id : 83.0 , trackster_id : 0.0 ,layer_id : 43.0.
Sequence cannt be generated for event_id : 83.0 , trackster_id : 0.0 ,layer_id : 45.0.
Sequence cannt be generated for event_id : 83.0 , trackster_id : 0.0 ,layer_id : 45.0.
Sequence cannt be generated for event_id : 83.0 , trackster_id : 0.0 ,layer_id : 47.0.
Sequence cannt be generated for event_id : 83.0 , trackster_id : 0.0 ,layer_id : 47.0.
Sequence cannt be generated for event_id : 84.0 , trackster_id : 0.0 ,layer_id : 36.0.
Sequence cannt be generated for event_id : 84.0 , trackster_id : 0.0 ,layer_id : 36.0.
Sequence cannt be generated for event_id : 

Sequence cannt be generated for event_id : 87.0 , trackster_id : 6.0 ,layer_id : 2.0.
Sequence cannt be generated for event_id : 87.0 , trackster_id : 6.0 ,layer_id : 2.0.
Sequence cannt be generated for event_id : 87.0 , trackster_id : 6.0 ,layer_id : 2.0.
Sequence cannt be generated for event_id : 87.0 , trackster_id : 6.0 ,layer_id : 4.0.
Sequence cannt be generated for event_id : 87.0 , trackster_id : 6.0 ,layer_id : 4.0.
Sequence cannt be generated for event_id : 87.0 , trackster_id : 7.0 ,layer_id : 10.0.
Sequence cannt be generated for event_id : 87.0 , trackster_id : 7.0 ,layer_id : 11.0.
Sequence cannt be generated for event_id : 87.0 , trackster_id : 7.0 ,layer_id : 11.0.
Sequence cannt be generated for event_id : 87.0 , trackster_id : 7.0 ,layer_id : 13.0.
Sequence cannt be generated for event_id : 87.0 , trackster_id : 7.0 ,layer_id : 13.0.
Sequence cannt be generated for event_id : 87.0 , trackster_id : 7.0 ,layer_id : 13.0.
Sequence cannt be generated for event_id : 87.0 

Sequence cannt be generated for event_id : 90.0 , trackster_id : 5.0 ,layer_id : 3.0.
Sequence cannt be generated for event_id : 90.0 , trackster_id : 5.0 ,layer_id : 3.0.
Sequence cannt be generated for event_id : 90.0 , trackster_id : 5.0 ,layer_id : 5.0.
Sequence cannt be generated for event_id : 90.0 , trackster_id : 5.0 ,layer_id : 5.0.
Sequence cannt be generated for event_id : 90.0 , trackster_id : 5.0 ,layer_id : 12.0.
Sequence cannt be generated for event_id : 90.0 , trackster_id : 5.0 ,layer_id : 12.0.
Sequence cannt be generated for event_id : 90.0 , trackster_id : 5.0 ,layer_id : 13.0.
Sequence cannt be generated for event_id : 91.0 , trackster_id : 0.0 ,layer_id : 34.0.
Sequence cannt be generated for event_id : 91.0 , trackster_id : 0.0 ,layer_id : 34.0.
Sequence cannt be generated for event_id : 91.0 , trackster_id : 0.0 ,layer_id : 34.0.
Sequence cannt be generated for event_id : 91.0 , trackster_id : 0.0 ,layer_id : 35.0.
Sequence cannt be generated for event_id : 91.0

Sequence cannt be generated for event_id : 96.0 , trackster_id : 1.0 ,layer_id : 30.0.
Sequence cannt be generated for event_id : 96.0 , trackster_id : 1.0 ,layer_id : 30.0.
Sequence cannt be generated for event_id : 96.0 , trackster_id : 1.0 ,layer_id : 30.0.
Sequence cannt be generated for event_id : 96.0 , trackster_id : 1.0 ,layer_id : 30.0.
Sequence cannt be generated for event_id : 96.0 , trackster_id : 1.0 ,layer_id : 32.0.
Sequence cannt be generated for event_id : 96.0 , trackster_id : 1.0 ,layer_id : 33.0.
Sequence cannt be generated for event_id : 96.0 , trackster_id : 1.0 ,layer_id : 34.0.
Sequence cannt be generated for event_id : 96.0 , trackster_id : 1.0 ,layer_id : 35.0.
Sequence cannt be generated for event_id : 96.0 , trackster_id : 1.0 ,layer_id : 36.0.
Sequence cannt be generated for event_id : 96.0 , trackster_id : 1.0 ,layer_id : 36.0.
Sequence cannt be generated for event_id : 96.0 , trackster_id : 1.0 ,layer_id : 37.0.
Sequence cannt be generated for event_id : 

Sequence cannt be generated for event_id : 100.0 , trackster_id : 0.0 ,layer_id : 35.0.
Sequence cannt be generated for event_id : 100.0 , trackster_id : 0.0 ,layer_id : 35.0.
Sequence cannt be generated for event_id : 100.0 , trackster_id : 0.0 ,layer_id : 37.0.
Sequence cannt be generated for event_id : 100.0 , trackster_id : 0.0 ,layer_id : 37.0.
Sequence cannt be generated for event_id : 100.0 , trackster_id : 0.0 ,layer_id : 38.0.
Sequence cannt be generated for event_id : 100.0 , trackster_id : 0.0 ,layer_id : 39.0.
Sequence cannt be generated for event_id : 100.0 , trackster_id : 0.0 ,layer_id : 40.0.
Sequence cannt be generated for event_id : 100.0 , trackster_id : 0.0 ,layer_id : 41.0.
Sequence cannt be generated for event_id : 100.0 , trackster_id : 0.0 ,layer_id : 41.0.
len(seq_df): 142065


In [50]:
immediate_layer_seq_store_25_Oct_len_5.head(15)

,index_original,sequence_id,event,trackster,purity,layer,E,eta,phi,x,y,z,nHits,trckEn,trckEta,trckPhi,trckType
0,0,1_0_1,1.0,0.0,0.0,1.0,0.077115,1.963233,0.158004,91.118462,14.518062,322.102753,5.0,57.759506,1.897144,0.196742,211.0
1,1,1_0_1,1.0,0.0,0.0,1.0,0.134952,1.931620,0.233432,92.770134,22.057596,322.102753,7.0,57.759506,1.897144,0.196742,211.0
2,9,1_0_1,1.0,0.0,0.0,2.0,0.104475,1.976022,0.226741,88.978676,20.528135,323.047272,3.0,57.759506,1.897144,0.196742,211.0
3,23,1_0_1,1.0,0.0,0.0,3.0,0.030137,1.963898,0.249134,90.181198,22.943840,325.072754,2.0,57.759506,1.897144,0.196742,211.0
4,32,1_0_1,1.0,0.0,0.0,3.0,0.027726,1.926386,0.244185,93.891945,23.393869,325.072754,2.0,57.759506,1.897144,0.196742,211.0
5,0,1_0_2,1.0,0.0,0.0,1.0,0.077115,1.963233,0.158004,91.118462,14.518062,322.102753,5.0,57.759506,1.897144,0.196742,211.0
6,1,1_0_2,1.0,0.0,0.0,1.0,0.134952,1.931620,0.233432,92.770134,22.057596,322.102753,7.0,57.759506,1.897144,0.196742,211.0
7,10,1_0_2,1.0,0.0,0.0,2.0,0.109699,1.941603,0.231176,92.128059,21.685528,323.047272,3.0,57.759506,1.897144,0.196742,211.0
8,23,1_0_2,1.0,0.0,0.0,3.0,0.030137,1.963898,0.249134,90.181198,22.943840,325.072754,2.0,57.759506,1.897144,0.196742,211.0
9,32,1_0_2,1.0,0.0,0.0,3.0,0.027726,1.926386,0.244185,93.891945,23.393869,325.072754,2.0,57.759506,1.897144,0.196742,211.0


In [51]:
# Lets save the sequence of length 5

immediate_layer_seq_store_25_Oct_len_5.to_csv('D:/CERN/data/singlepi_e100GeV_pu200_25Oct/L5_immediate_layer_seq_store_25_Oct.csv',index = False)

In [53]:
immediate_layer_seq_store_25_Oct[1::3]

,index_original,sequence_id,event,trackster,purity,layer,E,eta,phi,x,y,z,nHits,trckEn,trckEta,trckPhi,trckType
1,9,1_0_1,1.0,0.0,0.0,2.0,0.104475,1.976022,0.226741,88.978676,20.528135,323.047272,3.0,57.759506,1.897144,0.196742,211.0
4,10,1_0_2,1.0,0.0,0.0,2.0,0.109699,1.941603,0.231176,92.128059,21.685528,323.047272,3.0,57.759506,1.897144,0.196742,211.0
7,11,1_0_3,1.0,0.0,0.0,2.0,0.251543,1.947605,0.158610,92.875359,14.855724,323.047272,5.0,57.759506,1.897144,0.196742,211.0
10,12,1_0_4,1.0,0.0,0.0,2.0,0.021699,1.961069,0.337146,87.525536,30.680223,323.047272,2.0,57.759506,1.897144,0.196742,211.0
13,13,1_0_5,1.0,0.0,0.0,2.0,0.028932,1.986022,0.248850,87.588531,22.257795,323.047272,2.0,57.759506,1.897144,0.196742,211.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90310,32268,100_0_27,100.0,0.0,2.0,38.0,1.876165,1.864189,0.220444,129.188202,28.949202,416.748993,10.0,53.433231,1.863788,0.184085,211.0
90313,32269,100_0_28,100.0,0.0,2.0,39.0,0.707843,1.865875,0.194855,131.352676,25.923639,422.199005,9.0,53.433231,1.863788,0.184085,211.0
90316,32270,100_0_29,100.0,0.0,2.0,40.0,1.439834,1.883487,0.212144,130.149323,28.032238,427.649017,8.0,53.433231,1.863788,0.184085,211.0
90319,32271,100_0_30,100.0,0.0,2.0,41.0,0.652212,1.884759,0.208603,132.674957,28.085020,436.199005,3.0,53.433231,1.863788,0.184085,211.0


In [54]:
immediate_layer_seq_store_25_Oct_len_5[2::5]

,index_original,sequence_id,event,trackster,purity,layer,E,eta,phi,x,y,z,nHits,trckEn,trckEta,trckPhi,trckType
2,9,1_0_1,1.0,0.0,0.0,2.0,0.104475,1.976022,0.226741,88.978676,20.528135,323.047272,3.0,57.759506,1.897144,0.196742,211.0
7,10,1_0_2,1.0,0.0,0.0,2.0,0.109699,1.941603,0.231176,92.128059,21.685528,323.047272,3.0,57.759506,1.897144,0.196742,211.0
12,11,1_0_3,1.0,0.0,0.0,2.0,0.251543,1.947605,0.158610,92.875359,14.855724,323.047272,5.0,57.759506,1.897144,0.196742,211.0
17,12,1_0_4,1.0,0.0,0.0,2.0,0.021699,1.961069,0.337146,87.525536,30.680223,323.047272,2.0,57.759506,1.897144,0.196742,211.0
22,13,1_0_5,1.0,0.0,0.0,2.0,0.028932,1.986022,0.248850,87.588531,22.257795,323.047272,2.0,57.759506,1.897144,0.196742,211.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142042,32259,100_0_15,100.0,0.0,2.0,33.0,0.128122,1.829822,0.168508,126.469330,21.515116,389.499023,2.0,53.433231,1.863788,0.184085,211.0
142047,32260,100_0_16,100.0,0.0,2.0,33.0,0.933008,1.868665,0.198810,120.729805,24.323553,389.499023,3.0,53.433231,1.863788,0.184085,211.0
142052,32261,100_0_17,100.0,0.0,2.0,33.0,1.411881,1.861803,0.178185,122.081474,21.986248,389.499023,6.0,53.433231,1.863788,0.184085,211.0
142057,32262,100_0_18,100.0,0.0,2.0,34.0,3.147238,1.868138,0.178954,122.952545,22.240751,394.949005,16.0,53.433231,1.863788,0.184085,211.0
